In [ ]:
import numpy as np
import time
import scipy
import torch
from concurrent.futures import ProcessPoolExecutor

from old_gep_code.gep_problem import GEPProblem
from gep_config_parser import *
from old_gep_code.gep_primal_dual_main import prep_data



Running on cpu
Parsing the config file
Initializing the solver
Using Gurobi
Parsing the config file


In [2]:
CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"
SAMPLE_DURATION = 12
# SAMPLE_DURATION = 120

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

Parsing the config file


In [3]:
experiment_instance = experiment["experiments"][0]
data = prep_data(experiment_instance, shuffle=False, train=0.002, valid=0.002, test=0.996)

Wrangling the input data
Creating problem instance
Size of train set: 0
Size of val set: 0
Size of test set: 365
Size of mu: 573
Size of lambda: 72
Number of variables (size of y): 144
Number of inputs (size of X): 219


In [9]:
# ! My ALM functions:
def obj_fn_ALM(Yi):
    # Objective function evaluated for Y
    # return (
    #     0.5 * Yi.T @ self.Q_np @ Yi + self.p_np.T @ Yi
    # )  # Ensure Q_np: (100, 100), p_np: (100,)

    return data.obj_fn(torch.tensor(Yi).unsqueeze(0)).squeeze().numpy()

def eq_resid_ALM(Yi, Xi):
    # Equality residuals (X - A * Y)
    # return Xi - self.A_np @ Yi  # A_np: (50, 100), X: (50,), Y: (100,)
    return data.eq_resid(Xi, torch.tensor(Yi).unsqueeze(0)).squeeze().numpy()

def ineq_resid_ALM(Yi, Xi):
    # Inequality residuals (G * Y - h)
    # return self.G_np @ Yi - self.h_np  # G_np: (50, 100), h_np: (50,)
    return data.ineq_resid(Xi, torch.tensor(Yi).unsqueeze(0)).squeeze().numpy()

def L_rho(Yi, Xi, m, l, rho):
    # Compute inequality and equality residuals
    ineq_resid = ineq_resid_ALM(Yi, Xi)# Should be (50,)
    eq_resid = eq_resid_ALM(Yi, Xi)  # Should be (50,)

    # Augmented Lagrangian computation
    return (
        obj_fn_ALM(Yi)
        + m.T @ ineq_resid  # Linear term for inequality
        + l.T @ eq_resid  # Linear term for equality
        + (rho / 2)
        * (
            np.linalg.norm(np.maximum(ineq_resid, 0), 2) ** 2
            + np.linalg.norm(eq_resid, 2) ** 2
        )
    )

def solve_instance(i, Xi, tol=1e-4):
        """
        Solve ALM for a single instance.
        """
        mu_k = np.zeros(data.nineq)  # (50,)
        lamb_k = np.zeros(data.neq)  # (50,)
        rho = 1
        tau = 0.5
        alpha = 10
        K = 20

        epsilon = 1e-4
        rho_max = 5000
        # print(f"sample {i+1} of {833}")
        start_time = time.time()
        # Initial primal point
        Yi = np.random.uniform(-1, 1, data.ydim)  # Shape (100,)

        for k in range(K):
            print(f"{k+1}/{K}")
            # Minimize L_rho for current `mu_k` and `lambda_k`
            res = scipy.optimize.minimize(
                fun=L_rho,
                x0=Yi,
                args=(Xi, mu_k, lamb_k, rho),
                tol=tol,
                method="CG",
            )
            y_k = res.x  # Solution for this iteration, shape (100,)

            # Update dual variables using y_k
            mu_k = np.maximum(mu_k + rho * ineq_resid_ALM(y_k, Xi), 0)
            lamb_k = lamb_k + rho * eq_resid_ALM(y_k, Xi)

            # Calculate v_k to check for convergence
            inf_norm_eq_resid = np.max(np.abs(eq_resid_ALM(y_k, Xi)))
            sigma = np.maximum(ineq_resid_ALM(y_k, Xi), -mu_k / rho)
            inf_norm_sigma = np.max(np.abs(sigma))
            v_k = max(inf_norm_eq_resid, inf_norm_sigma)

            if v_k < epsilon:
                print(f"Converged for instance {i+1} at iteration {k+1}")
                break

            if k >= 1 and v_k > (tau * prev_v_k):
                rho = min(alpha * rho, rho_max)
            prev_v_k = v_k
            Yi = y_k  # Use the current solution as the initial guess for the next iteration
            print(obj_fn_ALM(Yi))
            print(v_k)

            rhs_eq = data.rhs_eq[0]
            rhs_ineq = data.rhs_ineq[0]

            # print(mu_k)

            # print(lamb_k)

            print((data.dual_obj_fn(torch.tensor(mu_k.tolist()).unsqueeze(0), torch.tensor(lamb_k.tolist()).unsqueeze(0), rhs_eq, rhs_ineq)).item())

        end_time = time.time()
        print(obj_fn_ALM(Yi))
        return y_k, mu_k, lamb_k, end_time - start_time

def ALM_solve(X, tol=1e-4):
    X_np = X.detach().cpu().numpy() if hasattr(X, "detach") else X

    # Parallel processing
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(solve_instance, range(len(X_np)), X_np))

    # Unpack results
    Y, times = zip(*results)
    total_time = sum(times)
    parallel_time = total_time / len(X_np)
    sols = np.array(Y)

    return sols, total_time, parallel_time

In [10]:
X = data.trainX
y, mu, lamb, time_taken = solve_instance(0, X, tol=0.01)

1/20


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 429 is different from 573)

In [ ]:
OPTIMAL_OBJ = 8.18648032e+08 # First sample

print(f"Optimality gap (%): {abs(obj_fn_ALM(y) - OPTIMAL_OBJ) / OPTIMAL_OBJ * 100}")
# print(mu)
# print(lamb)

g = ineq_resid_ALM(y, X)
h = eq_resid_ALM(y, X)

# print(g)
# print(mu[g < 0])

Optimality gap (%): 0.0008450506876519232


In [ ]:
def dual_obj_fn(mu, lamb, rhs_eq, rhs_ineq):
    """ Objective function of the dual problem.
    max -mu^T d - lamb^T b

    Args:
        mu (_type_): Dual variable for inequality constraints
        lamb (_type_): Dual variable for equality constraints
        rhs_ineq (_type_): Constants at the right-hand side of the inequality constraints
        rhs_eq (_type_): Constants at the right-hand side of the equality constraints

    Returns:
        _type_: Dual objective value
    """
    ineq_term = torch.sum(mu * rhs_ineq, dim=1)
    
    eq_term = torch.sum(lamb * rhs_eq, dim=1)

    return -(ineq_term + eq_term)

In [ ]:
rhs_eq = data.rhs_eq[0]
rhs_ineq = data.rhs_ineq[0]

print((data.dual_obj_fn(torch.tensor(mu.tolist()).unsqueeze(0), torch.tensor(lamb.tolist()).unsqueeze(0), rhs_eq, rhs_ineq)).item())

828113204.1399043
